In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2'

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data: Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentaion : bool 
    params_image_size : list 


@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path
    


In [6]:
from CNNproject.constants import * 
from CNNproject.utils.common import read_yaml, create_directories 
import tensorflow as tf 

In [7]:
class ConfigManager : 
    def __init__ (self, config_filepath = CONFIG_YAML, params_filepath = PARMAS_YAML) :
        self.config = read_yaml (config_filepath)
        self.params = read_yaml (params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config (self) -> PrepareCallbacksConfig : 
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) 
        create_directories([model_ckpt_dir,
                            Path(config.tensorboard_root_log_dir)])
        
        prepare_call_back_config = PrepareCallbacksConfig( 
            root_dir = Path (config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path (config.checkpoint_model_filepath)
        )
        print("pass")
        return prepare_call_back_config


    def get_training_config(self) -> TrainingConfig : 
        training_config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        create_directories ([Path(training_config.root_dir)])


        training_config= TrainingConfig( 
        root_dir =  Path(training_config.root_dir),
        trained_model_path = Path(training_config.trained_model_path),
        updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
        training_data =  Path (training_data),
        params_epochs = params.EPOCHS,
        params_batch_size = params.BATCH_SIZE,
        params_is_augmentaion = params.AUGMENTATION,
        params_image_size = params.IMAGE_SIZE,
        )
        print("pass")
        return training_config

In [8]:
import os 
import urllib.request as request 
from zipfile import ZipFile
import tensorflow as tf 
import time

class PrepareCallback:
    def __init__ (self, config: PrepareCallbacksConfig) :
        self.config = config

    
    @property
    def _create_tb_callbacks(self): 
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self): 
        return tf.keras.callbacks.ModelCheckpoint( 
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    

    def get_tb_ckpt_callbacks(self): 
        return (
            [self._create_ckpt_callbacks, 
             self._create_tb_callbacks]
        )

In [9]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time

In [10]:
class Training: 
    def __init__(self, config: TrainingConfig) : 
        self.config = config 
        tf.config.run_functions_eagerly(True)
    def get_base_model (self) :
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self) :
        datagenerator_kwargs = dict(rescale = 1./255, validation_split =0.20) #rescaling/validation/ augmentaion
        
        dataflow_kwargs = dict(                                                #resizing / batchsize
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(directory= self.config.training_data, subset= "validation", shuffle= False, **dataflow_kwargs )


        if self.config.params_is_augmentaion: 
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator( 
                rotation_range = 40, 
                horizontal_flip = True, 
                width_shift_range = 0.2, 
                height_shift_range = 0.2, 
                shear_range = 0.2, 
                zoom_range = 0.2, 
                ** datagenerator_kwargs
            )
        else :
            train_datagenerator = valid_datagenerator


        self.train_generator = train_datagenerator.flow_from_directory(directory= self.config.training_data, subset= "training", shuffle= True, **dataflow_kwargs )



    @staticmethod 
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def train (self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size 
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        print("Training samples:", self.train_generator.samples)
        print("Validation samples:", self.valid_generator.samples)

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list

        )

        self.save_model(
            path = self.config.trained_model_path, 
            model =self.model 
        )




In [ ]:
try : 
    config = ConfigManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepareCallbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepareCallbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e : 
    raise e 

[2025-02-07 02:47:28,151: INFO: common: yaml file config\config.yaml loadded successfully]
[2025-02-07 02:47:28,151: INFO: common: yaml file params.yaml loadded successfully]
[2025-02-07 02:47:28,151: INFO: common: Created directory at: artifacts]
[2025-02-07 02:47:28,151: INFO: common: Created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-02-07 02:47:28,151: INFO: common: Created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
pass
[2025-02-07 02:47:28,157: INFO: common: Created directory at: artifacts\training]
pass
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Training samples: 312
Validation samples: 78
Epoch 1/10


C:\Users\aamer\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 [==============================] - 75s 4s/step - loss: 13.6835 - accuracy: 0.5270 - val_loss: 4.0872 - val_accuracy: 0.4688
Epoch 2/10
19/19 [==============================] - 79s 4s/step - loss: 7.4114 - accuracy: 0.6216 - val_loss: 4.7639 - val_accuracy: 0.6094
Epoch 3/10
19/19 [==============================] - 100s 5s/step - loss: 3.1750 - accuracy: 0.7905 - val_loss: 2.4462 - val_accuracy: 0.7969
Epoch 4/10
19/19 [==============================] - 76s 4s/step - loss: 2.1185 - accuracy: 0.8480 - val_loss: 1.2037 - val_accuracy: 0.8438
Epoch 5/10
19/19 [==============================] - 75s 4s/step - loss: 0.6000 - accuracy: 0.9054 - val_loss: 1.6509 - val_accuracy: 0.8438
Epoch 6/10
19/19 [==============================] - 79s 4s/step - loss: 3.3487 - accuracy: 0.7939 - val_loss: 0.7087 - val_accuracy: 0.9375
Epoch 7/10
19/19 [==============================] - 79s 4s/step - loss: 3.6232 - accuracy: 0.7500 - val_loss: 2.3455 - val_accuracy: 0.7656
Epoch 8/10
19/19 [===========